<a href="https://colab.research.google.com/github/zsexton/homelab/blob/main/train-yolov9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOv9 on a Custom Dataset
---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/WongKinYiu/yolov9)
[![arXiv](https://img.shields.io/badge/arXiv-2402.13616-b31b1b.svg)](https://arxiv.org/pdf/2402.13616.pdf)

## Before you start / Setup

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Thu Mar 13 19:40:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os, glob
from IPython.display import Image
from google.colab import drive, userdata


HOME = os.getcwd()
YOLO = os.path.join(HOME, 'yolov9')
print(HOME)
print(YOLO)

/content
/content/yolov9


Connect the Notebook with your Google Drive to store the results

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Number the trainign runs consecutively

In [ ]:
Lauf_num = 'Lauf_7'

## Clone and Install

**NOTE**: These are the github commands for cloning, adding, commiting and pushing

In [ ]:
github_token = userdata.get('github_token')

# clone GitHub Repo
!git clone https://{github_token}@github.com/KoniHD/yolov9.git
!pip install -r {YOLO}/requirements.txt -q

## Download necessary data

### Download weights

**NOTE:** There are multiple more versions of pretrained weights provided by the authors of the paper. You will have to look into their differences! [repository](https://github.com/WongKinYiu/yolov9).

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-m.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-m.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-s2.pt

In [ ]:
!ls -la {HOME}/weights

### Download LOCO dataset

**NOTE:** This might need to be adjusted to deal with COCO style anotations

In [ ]:
%cd {YOLO}
!{YOLO}/scripts/get_loco.sh # This script downloads the dataset in the active directory
%cd {HOME}
!python {YOLO}/scripts/transform_annotations.py --d {YOLO}/loco --convert-images --convert-to-yolo

## Train custom model on LOCO dataset

**Note:** `train.py` is only for gelan models `train_dual.py` is only for yolov9 models

In [ ]:
!python {YOLO}/train_dual.py --batch 8 --epochs 50 --device 0 --min-items 0 --img 640 \
--data {YOLO}/loco.yaml \
--weights {HOME}/weights/yolov9-e.pt \
--cfg {YOLO}/models/detect/yolov9_custom.yaml \
--hyp hyp.scratch-high.yaml

## Detection with self-trained detection model

**Note:** There is no test set. Image `509189,8734.jpg` is just one example from the training dataset. It includes a lot of pallets and a pallet truck which should be detected.

In [ ]:
!python {YOLO}/detect_dual.py --weights {YOLO}/runs/train/exp/weights/best.pt --conf 0.1 --source {YOLO}/loco/images/val/509189,8734.jpg --device 0

In [ ]:
Image(filename=f"{YOLO}/loco/images/val/509189,8734.jpg", width=640)    # or maybe width=640???

In [ ]:
Image(filename=f"{YOLO}/runs/detect/exp/509189,8734.jpg", width=640)    # or maybe width=640???

## Saving the obtained weights to Drive

In [ ]:
!cp -r {YOLO}/runs/ {HOME}/drive/MyDrive/Studium/'Bachelor Arbeit'/runs/{Lauf_num}

## Examine Training Results

**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp2/results.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp2/confusion_matrix.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp2/val_batch0_pred.jpg", width=1000)

## Validate Custom Model

**NOTE**: Look into this! Why do you need validation?

In [ ]:
%cd {HOME}/yolov9

!python val_dual.py \
--img 640 --batch 8 --conf 0.001 --iou 0.7 --device 0 \
--data {YOLO}/loco.yaml \
--weights {YOLO}/runs/train/exp2/weights/best.pt

## BONUS: Deploy YOLOv9 Model with Inference

**NOTE:** To deploy the model and display inference results, we will need two additional packages - [`inference`](https://pypi.org/project/inference) and [`supervision`](https://pypi.org/project/supervision). Let's install and import them!

In [ ]:
image_paths = sv.list_files_with_extensions(
    directory=f"{dataset.location}/test/images",
    extensions=['png', 'jpg', 'jpeg']
)
image_path = random.choice(image_paths)
image = cv2.imread(image_path)

result = model.infer(image, confidence=0.1)[0]
detections = sv.Detections.from_inference(result)

**NOTE:** Finally, let's use supervision and [annotate](https://supervision.roboflow.com/develop/annotators/) our results.